In [26]:
import pandas as pd
import numpy as np

epa_dha = [621, 629]
omega_6 = [618, 619, 620]
food_list = pd.read_csv("data/food_codes_v3.csv", encoding="latin1")
food_nutrients = pd.read_parquet("data/dataset.parquet")
food_list["food_name"] = food_list.food_name.str.strip()
food_list["fdc_id"] = food_list.fdc_id.astype(int)
food_list["g"] = food_list["lbs"] * 453.592
food_list["price_per_100_g"] = food_list["price"] / food_list["g"] * 100

In [27]:
food_nutrients = food_nutrients.rename(columns={"name": "nutrient_name"})
food_nutrients["nutrient_nbr"] = food_nutrients.nutrient_nbr.astype(int)

In [29]:
list_nutrients = food_list.merge(food_nutrients, on="fdc_id", how="inner")
list_nutrients = list_nutrients.loc[list_nutrients.nutrient_nbr != 268]
list_nutrients["nutrient_density"] = (
    list_nutrients["amount"] / list_nutrients["price_per_100_g"]
)

# def get_combo_row(combo_name, unit_name, nutrient_nbrs, new_unit_nbr):
#     amounts = (
#         list_nutrients.loc[list_nutrients.nutrient_nbr.isin(nutrient_nbrs)]
#         .groupby("food")
#         .amount.sum()
#     )
#     densities = (
#         list_nutrients[["food", "price_per_100_g"]]
#         .drop_duplicates()
#         .set_index("food")
#         .join(amounts)
#     )
#     densities["nutrient_density"] = densities.amount / densities.price_per_100_g
#     densities["nutrient_nbr"] = new_unit_nbr
#     densities["nutrient_name"] = combo_name
#     densities["unit_name"] = unit_name
#     row = (
#         densities.reset_index()
#         .pivot(
#             index=["nutrient_nbr", "nutrient_name", "unit_name"],
#             columns="food",
#             values="nutrient_density",
#         )
#         .reset_index()
#         .set_index("nutrient_nbr")
#     )
#     return row


# epa_dpa_row = get_combo_row("EPA+DHA", "g", epa_dha, -1)
# omega_6_row = get_combo_row("Omega-6", "g", omega_6, -2)

In [53]:
# Pivot list_nutrients to prepare for merge with constraints
food_nutrient_densities = (
    list_nutrients.pivot(
        index=["food_name", "fdc_id", "price_per_100_g"],
        columns="nutrient_nbr",
        values="amount",
    )
    .fillna(0)
    .reset_index()
)
# lp = pd.concat([lp, epa_dpa_row])
# lp = pd.concat([lp, omega_6_row])
# lp = lp.loc[~lp.index.isin(epa_dha)]
# lp = lp.loc[~lp.index.isin(omega_6)]

In [55]:
food_nutrient_densities.to_csv("data/food_nutrient_amounts.csv")

In [32]:
# df_for_pulp = constraints.drop(columns=["nutrient_name", "unit_name"]).merge(
#     lp, on="nutrient_nbr", how="right"
# )
# df_for_pulp.to_csv("df_for_pulp_testing.csv")